# LSTM et GRU - Version Super Simple ! 😊

## 🤷‍♂️ Le problème avec les RNN normaux

Vous vous souvenez des RNN du notebook précédent ? Ils avaient un gros problème :

**Ils oublient vite !** 🤦‍♂️

### Exemple :
```
Phrase : "Ce film était nul au début mais en fait c'était génial"

RNN normal :
- Lit "Ce film était nul" → Pense "film nul"
- Lit "au début mais" → Commence à oublier "nul"
- Lit "en fait c'était génial" → Se souvient surtout de "génial"

Résultat : Ça marche... par chance ! 🍀

Mais si la phrase était plus longue, le RNN oublierait le début !
```

---

## 💡 La solution : LSTM et GRU

### LSTM = RNN avec une meilleure mémoire

**L'idée simple :** 
- ✅ Garder les infos importantes
- ❌ Oublier les infos inutiles
- 🎯 Décider quoi montrer en sortie

### GRU = Version simplifiée du LSTM

**L'idée :** Faire la même chose que LSTM mais plus simple !

---

### Analogie de la mémoire 🧠

Imaginez votre cerveau quand vous lisez :

**RNN normal :**
```
Mot 1: "Ce" → Je retiens "Ce"
Mot 2: "film" → J'oublie "Ce", je retiens "film"
Mot 3: "était" → J'oublie "film", je retiens "était"
...
```

**LSTM/GRU :**
```
Mot 1: "Ce" → Je retiens "Ce"
Mot 2: "film" → Je garde "Ce" ET j'ajoute "film"
Mot 3: "était" → Je garde ce qui est important et j'ajoute "était"
...
```

## 📦 Installation

In [ ]:
# Installation
!pip install tensorflow numpy matplotlib

In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

print("✅ Tout est prêt !")

## 🛠️ Créons nos 3 modèles

On va faire 3 modèles :
1. **RNN** - Le modèle de base (qui oublie)
2. **LSTM** - Avec bonne mémoire
3. **GRU** - Avec bonne mémoire mais plus simple

In [ ]:
def creer_rnn():
    """RNN normal (qui oublie vite)"""
    model = models.Sequential([
        layers.Embedding(1000, 50, input_length=20),  # Convertit mots en nombres
        layers.SimpleRNN(32),                         # RNN avec 32 neurones
        layers.Dense(1, activation='sigmoid')         # Sortie : positif ou négatif
    ])
    return model

def creer_lstm():
    """LSTM (bonne mémoire)"""
    model = models.Sequential([
        layers.Embedding(1000, 50, input_length=20),
        layers.LSTM(32),                              # LSTM au lieu de RNN
        layers.Dense(1, activation='sigmoid')
    ])
    return model

def creer_gru():
    """GRU (bonne mémoire, plus simple)"""
    model = models.Sequential([
        layers.Embedding(1000, 50, input_length=20),
        layers.GRU(32),                               # GRU au lieu de RNN
        layers.Dense(1, activation='sigmoid')
    ])
    return model

# Créer les modèles
rnn_model = creer_rnn()
lstm_model = creer_lstm()
gru_model = creer_gru()

print("✅ 3 modèles créés !")
print(f"RNN:  {rnn_model.count_params():,} paramètres")
print(f"LSTM: {lstm_model.count_params():,} paramètres")
print(f"GRU:  {gru_model.count_params():,} paramètres")

print("\n💡 Plus de paramètres = plus complexe mais potentiellement meilleur")

## 📝 Créons des données simples

On va faire des phrases où il faut bien se souvenir du début pour comprendre la fin.

In [ ]:
# Phrases où il faut se souvenir du début
phrases_positives = [
    "Super film même si long",
    "Excellent produit malgré prix élevé",
    "Génial service même avec attente",
    "Fantastique expérience malgré problèmes début",
    "Parfait résultat même si difficile",
    "Magnifique spectacle malgré salle bruyante",
    "Délicieux repas même si service lent",
    "Formidable cours malgré professeur strict",
    "Incroyable voyage même avec retards",
    "Merveilleux livre malgré pages nombreuses"
]

phrases_negatives = [
    "Nul film même si acteurs connus",
    "Mauvais produit malgré belle publicité",
    "Horrible service même avec sourires",
    "Décevant résultat malgré promesses",
    "Catastrophique expérience même si cher",
    "Ennuyeux spectacle malgré beau théâtre",
    "Dégoutant repas même si joli restaurant",
    "Inutile cours malgré bon professeur",
    "Raté voyage même avec beau temps",
    "Barbant livre malgré belle couverture"
]

# Combiner
toutes_phrases = phrases_positives + phrases_negatives
labels = [1] * len(phrases_positives) + [0] * len(phrases_negatives)  # 1=positif, 0=négatif

# Multiplier pour avoir plus de données
toutes_phrases = toutes_phrases * 10  # 200 phrases au total
labels = labels * 10

print(f"📊 {len(toutes_phrases)} phrases créées")
print(f"📊 {sum(labels)} positives, {len(labels)-sum(labels)} négatives")

print("\n📝 Exemples:")
print("Positif:", phrases_positives[0])
print("Négatif:", phrases_negatives[0])

print("\n💡 Remarquez: Le mot important est au DÉBUT de chaque phrase !")

In [ ]:
# Préparer les données pour l'entraînement
tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(toutes_phrases)

# Convertir en nombres
sequences = tokenizer.texts_to_sequences(toutes_phrases)
X = pad_sequences(sequences, maxlen=20)  # Toutes les phrases = 20 mots max
y = np.array(labels)

# Diviser train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"✅ Données prêtes !")
print(f"Entraînement: {len(X_train)} phrases")
print(f"Test: {len(X_test)} phrases")

## 🏁 Le grand test !

On va entraîner nos 3 modèles et voir qui est le meilleur !

In [ ]:
# Fonction simple pour entraîner un modèle
def entrainer_modele(model, nom):
    print(f"\n🚀 Entraînement {nom}...")
    
    # Préparer le modèle
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    # Entraîner
    history = model.fit(
        X_train, y_train,
        epochs=5,           # Seulement 5 fois pour aller vite
        batch_size=32,
        validation_split=0.2,
        verbose=0           # Pas d'affichage détaillé
    )
    
    # Tester
    score = model.evaluate(X_test, y_test, verbose=0)
    precision = score[1]  # L'accuracy
    
    print(f"✅ {nom}: {precision:.1%} de précision")
    return precision, history

# Entraîner les 3 modèles
resultats = {}

precision_rnn, hist_rnn = entrainer_modele(rnn_model, "RNN")
resultats["RNN"] = precision_rnn

precision_lstm, hist_lstm = entrainer_modele(lstm_model, "LSTM") 
resultats["LSTM"] = precision_lstm

precision_gru, hist_gru = entrainer_modele(gru_model, "GRU")
resultats["GRU"] = precision_gru

print("\n🏆 RÉSULTATS FINAUX:")
print("=" * 30)
for modele, precision in sorted(resultats.items(), key=lambda x: x[1], reverse=True):
    print(f"{modele}: {precision:.1%}")

# Trouver le gagnant
gagnant = max(resultats, key=resultats.get)
print(f"\n🥇 Le gagnant est: {gagnant} !")

In [ ]:
# Graphique simple des résultats
noms = list(resultats.keys())
scores = list(resultats.values())
couleurs = ['red', 'blue', 'green']

plt.figure(figsize=(8, 5))
barres = plt.bar(noms, scores, color=couleurs, alpha=0.7)

# Ajouter les pourcentages sur les barres
for barre, score in zip(barres, scores):
    plt.text(barre.get_x() + barre.get_width()/2, barre.get_height() + 0.01, 
             f'{score:.1%}', ha='center', fontweight='bold', fontsize=12)

plt.title('🏆 Comparaison des modèles', fontsize=16, fontweight='bold')
plt.ylabel('Précision')
plt.ylim(0, 1)
plt.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("💡 Plus la barre est haute, mieux c'est !")

## 🧪 Test sur de nouvelles phrases

Testons nos modèles sur des phrases qu'ils n'ont jamais vues !

In [ ]:
# Fonction pour prédire
def predire(phrase, model):
    # Convertir la phrase en nombres
    sequence = tokenizer.texts_to_sequences([phrase])
    sequence_paddee = pad_sequences(sequence, maxlen=20)
    
    # Prédire
    score = model.predict(sequence_paddee, verbose=0)[0, 0]
    
    if score > 0.5:
        return "😊 POSITIF", score
    else:
        return "😞 NÉGATIF", 1-score

# Phrases de test
phrases_test = [
    "Superbe film même si un peu long",          # Positif
    "Affreux restaurant malgré belle décoration", # Négatif  
    "Parfait weekend même avec pluie",           # Positif
    "Horrible voyage malgré beau temps"          # Négatif
]

vrais_sentiments = ["😊 POSITIF", "😞 NÉGATIF", "😊 POSITIF", "😞 NÉGATIF"]

print("🧪 TEST SUR NOUVELLES PHRASES")
print("=" * 50)

for i, phrase in enumerate(phrases_test):
    print(f"\n📝 Phrase: '{phrase}'")
    print(f"   Vrai sentiment: {vrais_sentiments[i]}")
    print("   Prédictions:")
    
    # Tester avec chaque modèle
    for nom, model in [("RNN", rnn_model), ("LSTM", lstm_model), ("GRU", gru_model)]:
        sentiment, confiance = predire(phrase, model)
        
        # Vérifier si c'est correct
        if sentiment == vrais_sentiments[i]:
            resultat = "✅"
        else:
            resultat = "❌"
        
        print(f"     {nom:4s}: {sentiment} ({confiance:.1%}) {resultat}")

## 📊 Pourquoi LSTM/GRU sont meilleurs ?

### Visualisation simple :

In [ ]:
# Explication visuelle simple
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# RNN normal
ax1.set_title('🔴 RNN Normal', fontsize=14, fontweight='bold')
mots = ['Super', 'film', 'même', 'si', 'long']
memoire_rnn = [1.0, 0.7, 0.4, 0.2, 0.1]  # La mémoire diminue

ax1.bar(mots, memoire_rnn, color='red', alpha=0.7)
ax1.set_ylabel('Mémoire du mot "Super"')
ax1.set_ylim(0, 1)
ax1.text(2, 0.8, '😰 Oublie\n"Super"!', ha='center', fontsize=12, 
         bbox=dict(boxstyle="round", facecolor='white', alpha=0.8))

# LSTM/GRU
ax2.set_title('🔵 LSTM/GRU', fontsize=14, fontweight='bold')
memoire_lstm = [1.0, 0.9, 0.9, 0.8, 0.8]  # La mémoire reste

ax2.bar(mots, memoire_lstm, color='blue', alpha=0.7)
ax2.set_ylabel('Mémoire du mot "Super"')
ax2.set_ylim(0, 1)
ax2.text(2, 0.8, '😊 Se souvient\nde "Super"!', ha='center', fontsize=12,
         bbox=dict(boxstyle="round", facecolor='white', alpha=0.8))

plt.tight_layout()
plt.show()

print("💡 C'est pour ça que LSTM/GRU sont meilleurs sur nos phrases !")
print("   Ils se souviennent du mot important du début.")

## 🎯 Résumé Super Simple

### Ce qu'on a appris :

1. **RNN normal** 🔴 :
   - ✅ Simple et rapide
   - ❌ Oublie vite le début des phrases
   - 👍 Bon pour phrases courtes

2. **LSTM** 🔵 :
   - ✅ Se souvient bien du début
   - ✅ Très bon sur phrases longues
   - ❌ Un peu plus lent

3. **GRU** 🟢 :
   - ✅ Se souvient bien (comme LSTM)
   - ✅ Plus simple que LSTM
   - ✅ Plus rapide que LSTM

### 💡 Conseil simple :

**Utilisez GRU !** 🎯

C'est le meilleur compromis :
- Aussi bon que LSTM
- Plus simple
- Plus rapide

### 🚀 Dans la vraie vie :

- **Phrases courtes** (tweets) → RNN suffit
- **Phrases longues** (articles) → GRU ou LSTM
- **Pas sûr** → Commencez par GRU !

---

## 🎉 Félicitations !

Vous savez maintenant :
- ✅ Pourquoi RNN oublient
- ✅ Comment LSTM/GRU résolvent le problème
- ✅ Quand utiliser quoi
- ✅ Comment les coder avec TensorFlow

**Vous êtes prêts pour la suite ! 🚀**